# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [21]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import pickle

In [23]:
# load data from database
engine = create_engine('sqlite:///disaster_project.db')
df = pd.read_sql_table('disaster_response', engine)

In [25]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
X = df['message'].values
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

In [29]:
X.shape, Y.shape

((26216,), (26216, 36))

### 2. Write a tokenization function to process your text data

In [31]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet', 'stopwords'])
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
def tokenize(text):
    #text = re.sub('[^a-z0-9]+', ' ', text)
    #text = text.translate(str.maketrans('', '', string.punctuation)).lower().strip()
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for t in tokens:
        clean_t = lemmatizer.lemmatize(t).lower().strip()
        clean_tokens.append(clean_t)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [34]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
     ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=3)))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3, random_state=42)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [36]:
Y_pred = pipeline.predict(X_test)

In [37]:
from sklearn.metrics import classification_report
columns = Y.columns.values

In [38]:
for i, c in enumerate(columns):
    print(classification_report(Y_test.loc[:,c],Y_pred[:,i]))

             precision    recall  f1-score   support

          0       0.52      0.33      0.40      1873
          1       0.81      0.91      0.85      5934
          2       0.44      0.14      0.21        58

avg / total       0.74      0.76      0.74      7865

             precision    recall  f1-score   support

          0       0.89      0.95      0.92      6533
          1       0.61      0.42      0.50      1332

avg / total       0.84      0.86      0.85      7865

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

             precision    recall  f1-score   support

          0       0.72      0.77      0.74      4646
          1       0.63      0.57      0.60      3219

avg / total       0.68      0.69      0.68      7865

             precision    recall  f1-score   support

          0       0.93      0.98 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [39]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f54527ae1e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=3, n_jobs=1,
               oob_score=False, random_state=None, 

In [40]:
parameters = {'clf__estimator__max_depth': [8, 18],
              'clf__estimator__min_samples_split': [2, 5],  
              'clf__estimator__n_estimators': [14, 100]
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [41]:
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__max_depth': [8, 18], 'clf__estimator__min_samples_split': [2, 5], 'clf__estimator__n_estimators': [14, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [42]:
Y_pred = cv.predict(X_test)

In [43]:
for i, c in enumerate(columns):
    print(classification_report(Y_test.loc[:,c],Y_pred[:,i]))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1873
          1       0.75      1.00      0.86      5934
          2       0.00      0.00      0.00        58

avg / total       0.57      0.75      0.65      7865

             precision    recall  f1-score   support

          0       0.83      1.00      0.91      6533
          1       0.00      0.00      0.00      1332

avg / total       0.69      0.83      0.75      7865

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

             precision    recall  f1-score   support

          0       0.61      1.00      0.75      4646
          1       0.98      0.06      0.12      3219

avg / total       0.76      0.62      0.49      7865

             precision    recall  f1-score   support

          0       0.92      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [44]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
     ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
                    ])

In [45]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [46]:
Y_pred = pipeline.predict(X_test)

In [47]:
for i, c in enumerate(columns):
    print(classification_report(Y_test.loc[:,c],Y_pred[:,i]))

             precision    recall  f1-score   support

          0       0.56      0.23      0.32      1873
          1       0.79      0.94      0.86      5934
          2       0.27      0.12      0.17        58

avg / total       0.73      0.77      0.73      7865

             precision    recall  f1-score   support

          0       0.91      0.97      0.94      6533
          1       0.78      0.55      0.64      1332

avg / total       0.89      0.90      0.89      7865

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

             precision    recall  f1-score   support

          0       0.75      0.86      0.80      4646
          1       0.75      0.59      0.66      3219

avg / total       0.75      0.75      0.74      7865

             precision    recall  f1-score   support

          0       0.94      0.98 

### 9. Export your model as a pickle file

In [48]:
pickle.dump(cv, open('model_ml.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.